In [3]:
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim
from shapely.geometry import Point
import os

In [ ]:
## load our clean file 
df_coi = pd.read_csv('/Users/manmohitsingh/Desktop/Analyst Builder/My Projects/SM/2_subdomains/final_2_subdomains_ca.csv', dtype={'geoid20': str})

## load the tracts file
tracts = gpd.read_file ('/Users/manmohitsingh/Desktop/Analyst Builder/My Projects/SM/tl_2020_06_tract/tl_2020_06_tract.shp')\
                        .loc[:, ['GEOID', 'geometry']]\
                        .rename(columns={'GEOID': 'geoid20'})
                        
df_coi['geoid20'] = (
    df_coi['geoid20']
      .str.strip()      # remove stray spaces
      .str.zfill(11)    # pad with leading zeros to length 11
)

# 3) Quick check
print(df_coi['geoid20'].head(10))
print(df_coi['geoid20'].str.len().value_counts())




geolocator = Nominatim(user_agent = 'coi-geocoder')

def geocode_address(address):
    # Don’t attempt to geocode empty or nan
    if pd.isna(address) or not isinstance(address, str) or not address.strip():
        raise ValueError("Empty or invalid address")
    
    try:
        
        loc = geolocator.geocode(address, timeout=10)
    except (GeocoderTimedOut, GeocoderServiceError) as e:
        raise ValueError(f"Geocoding service error: {e}")

    if loc is None:
        raise ValueError("Address not found")
    return loc.latitude, loc.longitude


def lookup_geoid20(lat, lon):
    pt = gpd.GeoDataFrame(
        [{"geometry": Point(lon, lat)}],
        crs="EPSG:4326"
    ).to_crs(tracts.crs)
    joined = gpd.sjoin(pt, tracts, how="left", predicate="within")
    geoid = joined.iloc[0]["geoid20"]
    if pd.isna(geoid):
        raise ValueError("No tract found")
    return geoid


# 2a. Read  working Excel file
excel_path = '/Users/manmohitsingh/Desktop/Analyst Builder/My Projects/SM/COI.xlsx'
df = pd.read_excel(excel_path, dtype=str)

# 2b. Ensure geoid20 column exists and fill it 
if 'geoid20' not in df.columns:
    df['geoid20'] = ''

# 2c. List of COI columns to autofill
coi_cols = [
    'Early Childhood Education',  # Early Childhood Education
    'Elementary Education',  # Elementary Education
    'Secondary and Post-Secondary Education',  # Secondary & Post‑Secondary Ed
    'Educational Resources',  # Educational Resources
    'Pollution',  # Pollution
    'Healthy Enviornments',  # Healthy Environments
    'Safety-Related Resources',  # Safety‑Related Resources
    'Health Resources',  # Health Resources
    'Employment',  # Employment
    'Economic Resources',  # Economic Resources
    'Concentrated Socio-Economic Inequity',  # Concentrated Socio‑Economic Inequity
    'Housing Resources',  # Housing Resources
    'Social Resources',  # Social Resources
    'Wealth',  # Wealth
]
for col in coi_cols:
    if col not in df.columns:
        df[col] = ''      # add empty columns if missing

# 2d. Identify rows needing processing
mask = df['geoid20'].isna() | (df['geoid20']=='')
to_process = df.loc[mask, :].copy()
print(f"{len(to_process)} addresses to process")


from tqdm.notebook import tqdm

errors = []


for idx, row in tqdm(df.loc[df['geoid20'].isna()].iterrows(), total=df['geoid20'].isna().sum()):
    addr = row['Address']  
    try:
        lat, lon = geocode_address(addr)
        geoid = lookup_geoid20(lat, lon)

        ## lookup coi dataframe
        coi_row = df_coi.loc[df_coi['geoid20']==geoid]
        if coi_row.empty:
            raise ValueError(f"No COI data for {geoid}")
        coi_row = coi_row.iloc[0]

        # write results back
        df.at[idx, 'geoid20'] = geoid
        for col in coi_cols:
            df.at[idx, col] = coi_row[col]

    except Exception as e:
        # record the error and move on
        errors.append((idx, addr, str(e)))
        df.at[idx, 'Error'] = str(e)

print("Done. Errors:", errors)


out_path = '/Users/manmohitsingh/Desktop/Analyst Builder/My Projects/SM/COI_filled1.xlsx'
cols_to_drop = [c for c in df.columns if c.endswith('.1')]
df = df.drop(columns=cols_to_drop)
df.to_excel(out_path, index=False)
print("Saved:", out_path)

0    06001400100
1    06001400100
2    06001400100
3    06001400100
4    06001400100
5    06001400100
6    06001400100
7    06001400100
8    06001400100
9    06001400100
Name: geoid20, dtype: object
geoid20
11    90984
Name: count, dtype: int64
28 addresses to process


  0%|          | 0/28 [00:00<?, ?it/s]

Done. Errors: [(0, nan, 'Empty or invalid address'), (1, '1309 Broadway Unit A Alameda, CA 94501', 'Address not found'), (5, 'address NA ', 'No tract found'), (6, '39802 Cedar Blvd Apt. 202 Newark 94560 ', 'Address not found'), (9, 'Address NA ', 'No tract found'), (11, 'address NA ', 'No tract found'), (12, 'address NA ', 'No tract found'), (13, 'address NA ', 'No tract found'), (14, nan, 'Empty or invalid address'), (15, nan, 'Empty or invalid address'), (16, 'address NA ', 'No tract found'), (17, nan, 'Empty or invalid address'), (21, nan, 'Empty or invalid address')]
Saved: /Users/manmohitsingh/Desktop/Analyst Builder/My Projects/SM/COI_filled1.xlsx
